# Name : Joshua O'Curry ,     Student Number : 20235838
# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

## Task 1 approach 
1. First I wanted to iterate though the holbrook.txt line by line 
- I knew that mispellings would only be present on lines that had 'mispelling|correction'
- I could then search each of these lines for the common character '|' 
- The next step was to tokenize the line into words. I decided to create two seperate lists ('original' & 'corrected').
- I iterated through each word until once again I located the '|' substring.
- At this point all that was left to do was to get the index of this token in the sentence, use the first half in my 'original' list and the second half in my 'corrected' list
- To get the index I simply appended that index of the element the contained '|' to an empty list 'indexes = []'
- To seperate the word I used word.split() using '|' as the delimiter. The first element of this output was the mispelling whereas the second was the correction.
- I then replaced the 'misplelling|correction' token in original & corrected lists with the relevant token for each.
- I created a dictionary 'x' for each line which had three keys : 'original', 'corrected' and 'indexes'.
- Finally, still inside the for loop for each line I then appended each lines dictionary into the empty list data.
- The cell runs including the assert => the output generated is as expected.

In [1]:
lines = open("holbrook.txt").readlines()
data = []

# Write your code here

import nltk

for line in lines:
    indexes = []
    
    if '|' in line:
        tokens = nltk.word_tokenize(line.strip())
        original = list(tokens)
        corrected = list(tokens)
        
        for word in tokens:
            
            if '|' in word:
                indexes.append(tokens.index(word))
                
                original[original.index(word)] = word.split('|')[0]
                
                corrected[corrected.index(word)] = word.split('|')[1]
                
            else:
                pass
            
    else:
        
        original = nltk.word_tokenize(line.strip())
        corrected = list(original)
    x = {'original' : original, 'corrected': corrected, 'indexes' : indexes}   

    data.append(x)
    
    
data[:10]

assert(data[2] == {
    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
    'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [2]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

## Task 2 Approach
__NOTE__ : I found it easier to use a nested for loop rather than the Counter from collections as I was iterating word by word.
### Unigram Function
1. I began the unigram() function by creating an empty list to record every single word (including duplicates) in my corpus using a nested for loop.
- I then called the FreDist function from nltk on this new list.
- I returned this frequency distribution filtering for the inputted word which in the assert() was 'me'

### Prob Function
1. Similarily I began the prob function by generating this list of the complete vocabulary in the corpus( including duplicates).
- the length of this list was to amount of tokens in the corpus.
- I called my unigram() function on the inputted word and then divided the output by the above length of my new list so that I would get the probability of that word in this corpus.

In [113]:
from collections import Counter

def unigram(word):
    # Write your code here.
    vocab = []
    for el in train:
        for char in el['corrected']:
            vocab.append(char.lower())
    freq_dist = nltk.FreqDist(vocab)
    return freq_dist[word]

def prob(word):
    # Write your code here.
    vocab = []
    for el in train:
        for char in el['corrected']:
            vocab.append(char.lower())
    
    return unigram(word)/len(vocab)

# Test your code with the following

assert(unigram("me")==87)

print(unigram('me'))
print(prob('me'))

87
0.003989178779402999


## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [8]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

## Task 3 approach
1. In the same way as the previous task I started by creating a list of the complete vocabulary of the corpus.
- I then created a set from this list to ensure I got all of the unique tokens.
- Next, I iterated through this set of unique words calculating the edit_distance between each and the inputted word.
- I initialised the min_dist variable as a negative as edit distance will always be >= 0 and so the first condition is that if min_dist is < 0 overwrite it. This just always stores the edit distance between the input and the first word.
- I then continue checking every other word only overwriting min_dist if it will make min_dist lower.
- Once I have the minimum distance stored I then repeat this iteration to get all of the distances however, this time I will be appending the word from the unique list to a candidates list if its distance matches the minimum distance.

In [110]:
def get_candidates(token):
    # Write your code here.
    vocab = []
    close_tokens = []
    
    min_dist = -1
    
    for el in train:
        for char in el['corrected']:
            vocab.append(char.lower())
                
    vocab_unique = set(vocab)
    
    
    for word in vocab_unique:
        distance = edit_distance(token, word)
        
        if min_dist < 0:
            min_dist = distance
        else:
            
            if distance < min_dist:
                min_dist = distance
            else:
                pass
        
    for word in vocab_unique:
        distance = edit_distance(token, word)
        
        if distance == min_dist:
            close_tokens.append(word)
        
    return close_tokens
        


# Test your code as follows
assert get_candidates("minde") == ['mind', 'mine']
get_candidates("minde")

['mind', 'mine']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


## Task 4 approach
1. The first thing to note for this function is that the input is essentially a sentence in the form of tokens.
-  We start by creating an empty list which will be the list that I return in the end of the function.
- Then, I created a set of all the unique tokens in the corpus.
- I will then iterate over each word in the inputted sentence.
- If the word is present in the unique vocabulary set it will immediately be stored in the corrected list.
- If it is not, we apply the get_candidates function.
- If it is a mispelling or not present in the corpus, we can expect a number of candidates to be returned.
- We then iterate over each of these candidates calling the Prob() function on each and storing their probability in a list.
- Next, we get the index of the maximum probability and append the candidate with the corresponding index to the corrected list.  
***
__MODIFICATION__ I added a counter _n_ which let me know how far along my algorithm was, I found this useful in future aspects of the project.

In [111]:
def correct(sentence):
    # Write your code here
    corrected = []
    n = 0
    vocab = []
    
    for el in train:
        for char in el['corrected']:
            vocab.append(char.lower())
                
    vocab_unique = list(set(vocab))
    #tokens = nltk.word_tokenize(sentence)
    for token in sentence:
        probs = []
        token = token.lower()
        if token in vocab_unique:
            corrected.append(token)
        else:
            candidates = get_candidates(token)
            for word in candidates:
                probs.append(prob(word))
            corrected.append(candidates[probs.index(max(probs))])
            
        n += 1
        print(f'{n} word(s) corrected')
    return corrected


assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   
correct(["this","whitr","cat"])

1 word(s) corrected


NameError: name 'prob' is not defined

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

##  Task 5 Approach

1. The first thing to do was to create all of the empty list I would be needing throughout this algorithm.
 - correction is a list of all the corrected words generated from the correct() function.
 - false is a list of all of the words in the 'original' key for each line. (This will be the input for correct()).
 - true is a list of all of the words in the 'corrected' key for each line.(This will be compared against correction)
 - pairs will be true and correction zipped together.
 - match will be a boolean list of 1s, and 0s indicating matches or not respectively.( similar for match_unique)  
  
- Next I created the false list by iterating through each word of the 'original' keys in the data and appending them to false.
- I then called the correct function on this list the generate the correction list.
- Similarily to how I created a false list I also created a true list.
- I then zipped both the true list and the correction list together which returns a list of tuples in which the first element of each is what the value should be and the second value of each is what my correct function deemed it to be.
- The next step was to get a unique set of these pairs which I did using set() . (Note: I changed this element back to a list in the case that I would want to do further analysis on it and it would keep its ordering).
- Next, I iterated through each tuple comparing the elements and if they were equivalent I would append a 1 to match and if not I would append a 0 to match.
- The final step was to get the accuracy of the model and I did this simply by summing the match list which gives me the number of correct predictions and dividing it by the length of matches which returns the number of predictions.  
__NOTE__ I  have defined accuracy as :  
$$\frac{Number of Correct Predictions}{Total Number of Predictions}$$
***
__MODIFICATION__ in conjunction with the modification of the correct() function I also included a print() which tells the length of the false string (the number of words to run through the correct() function).
***
My model yielded an accuracy of about 84% when considering every word including duplicates and 67% when excluding duplicates.

In [114]:
def accuracy(test):
    # Write your code here
    correction = []
    false = []
    true =[]
    pairs = []
    match = []
    match_unique = []
    
    
    for el in test:
        for char in el['original']:
            false.append(char.lower())
    print(f'the length of the false string is {len(false)}')
    
            
    correction = correct(false)
    
    
    for el in test:
        for char in el['corrected']:
            true.append(char.lower())
    
            
    zipped = zip(true,correction)
    for pair in zipped:
        pairs.append(pair)
    #print(pairs)
    
    for pair in pairs:
        if pair[0] == pair[1]:
            match.append(1)
        else:
            match.append(0)        
    
    pairs_unique = set(pairs)
        
    for pair in pairs_unique:
        if pair[0] == pair[1]:
            match_unique.append(1)
        else:
            match_unique.append(0)
        
    
    
    print(f'The accuracy of this model when counting every word is {(sum(match)/len(match)) * 100}%')
    print(f'The accuracy of this model when counting only unique words is {(sum(match_unique)/len(match_unique)) * 100}%')

accuracy(test)

the length of the false string is 1136
1 word(s) corrected
2 word(s) corrected
3 word(s) corrected
4 word(s) corrected
5 word(s) corrected
6 word(s) corrected
7 word(s) corrected
8 word(s) corrected
9 word(s) corrected
10 word(s) corrected
11 word(s) corrected
12 word(s) corrected
13 word(s) corrected
14 word(s) corrected
15 word(s) corrected
16 word(s) corrected
17 word(s) corrected
18 word(s) corrected
19 word(s) corrected
20 word(s) corrected
21 word(s) corrected
22 word(s) corrected
23 word(s) corrected
24 word(s) corrected
25 word(s) corrected
26 word(s) corrected
27 word(s) corrected
28 word(s) corrected
29 word(s) corrected
30 word(s) corrected
31 word(s) corrected
32 word(s) corrected
33 word(s) corrected
34 word(s) corrected
35 word(s) corrected
36 word(s) corrected
37 word(s) corrected
38 word(s) corrected
39 word(s) corrected
40 word(s) corrected
41 word(s) corrected
42 word(s) corrected
43 word(s) corrected
44 word(s) corrected
45 word(s) corrected
46 word(s) corrected
47 w

382 word(s) corrected
383 word(s) corrected
384 word(s) corrected
385 word(s) corrected
386 word(s) corrected
387 word(s) corrected
388 word(s) corrected
389 word(s) corrected
390 word(s) corrected
391 word(s) corrected
392 word(s) corrected
393 word(s) corrected
394 word(s) corrected
395 word(s) corrected
396 word(s) corrected
397 word(s) corrected
398 word(s) corrected
399 word(s) corrected
400 word(s) corrected
401 word(s) corrected
402 word(s) corrected
403 word(s) corrected
404 word(s) corrected
405 word(s) corrected
406 word(s) corrected
407 word(s) corrected
408 word(s) corrected
409 word(s) corrected
410 word(s) corrected
411 word(s) corrected
412 word(s) corrected
413 word(s) corrected
414 word(s) corrected
415 word(s) corrected
416 word(s) corrected
417 word(s) corrected
418 word(s) corrected
419 word(s) corrected
420 word(s) corrected
421 word(s) corrected
422 word(s) corrected
423 word(s) corrected
424 word(s) corrected
425 word(s) corrected
426 word(s) corrected
427 word(s

758 word(s) corrected
759 word(s) corrected
760 word(s) corrected
761 word(s) corrected
762 word(s) corrected
763 word(s) corrected
764 word(s) corrected
765 word(s) corrected
766 word(s) corrected
767 word(s) corrected
768 word(s) corrected
769 word(s) corrected
770 word(s) corrected
771 word(s) corrected
772 word(s) corrected
773 word(s) corrected
774 word(s) corrected
775 word(s) corrected
776 word(s) corrected
777 word(s) corrected
778 word(s) corrected
779 word(s) corrected
780 word(s) corrected
781 word(s) corrected
782 word(s) corrected
783 word(s) corrected
784 word(s) corrected
785 word(s) corrected
786 word(s) corrected
787 word(s) corrected
788 word(s) corrected
789 word(s) corrected
790 word(s) corrected
791 word(s) corrected
792 word(s) corrected
793 word(s) corrected
794 word(s) corrected
795 word(s) corrected
796 word(s) corrected
797 word(s) corrected
798 word(s) corrected
799 word(s) corrected
800 word(s) corrected
801 word(s) corrected
802 word(s) corrected
803 word(s

1128 word(s) corrected
1129 word(s) corrected
1130 word(s) corrected
1131 word(s) corrected
1132 word(s) corrected
1133 word(s) corrected
1134 word(s) corrected
1135 word(s) corrected
1136 word(s) corrected
The accuracy of this model when counting every word is 83.97887323943662%
The accuracy of this model when counting only unique words is 67.29222520107238%


## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


## Task 6 Approach

I believe the first step in improving the model is seeing what words are being misclassified and then seeking ways the quantity of these errors can be minimized. Firstly, I am going to deconstruct the above accuracy function in order to end up with a tuple of three elements ('original', 'correct', 'corrected_by_me').

In [52]:
false = []
for el in test:
    for char in el['original']:
        false.append(char.lower())
        
correction = correct(false)

true= []
for el in test:
    for char in el['corrected']:
        true.append(char.lower())
        
trips = []
zipped = zip(true, false ,correction)
for trip in zipped:
    trips.append(trip)

1 word(s) corrected
2 word(s) corrected
3 word(s) corrected
4 word(s) corrected
5 word(s) corrected
6 word(s) corrected
7 word(s) corrected
8 word(s) corrected
9 word(s) corrected
10 word(s) corrected
11 word(s) corrected
12 word(s) corrected
13 word(s) corrected
14 word(s) corrected
15 word(s) corrected
16 word(s) corrected
17 word(s) corrected
18 word(s) corrected
19 word(s) corrected
20 word(s) corrected
21 word(s) corrected
22 word(s) corrected
23 word(s) corrected
24 word(s) corrected
25 word(s) corrected
26 word(s) corrected
27 word(s) corrected
28 word(s) corrected
29 word(s) corrected
30 word(s) corrected
31 word(s) corrected
32 word(s) corrected
33 word(s) corrected
34 word(s) corrected
35 word(s) corrected
36 word(s) corrected
37 word(s) corrected
38 word(s) corrected
39 word(s) corrected
40 word(s) corrected
41 word(s) corrected
42 word(s) corrected
43 word(s) corrected
44 word(s) corrected
45 word(s) corrected
46 word(s) corrected
47 word(s) corrected
48 word(s) corrected
4

382 word(s) corrected
383 word(s) corrected
384 word(s) corrected
385 word(s) corrected
386 word(s) corrected
387 word(s) corrected
388 word(s) corrected
389 word(s) corrected
390 word(s) corrected
391 word(s) corrected
392 word(s) corrected
393 word(s) corrected
394 word(s) corrected
395 word(s) corrected
396 word(s) corrected
397 word(s) corrected
398 word(s) corrected
399 word(s) corrected
400 word(s) corrected
401 word(s) corrected
402 word(s) corrected
403 word(s) corrected
404 word(s) corrected
405 word(s) corrected
406 word(s) corrected
407 word(s) corrected
408 word(s) corrected
409 word(s) corrected
410 word(s) corrected
411 word(s) corrected
412 word(s) corrected
413 word(s) corrected
414 word(s) corrected
415 word(s) corrected
416 word(s) corrected
417 word(s) corrected
418 word(s) corrected
419 word(s) corrected
420 word(s) corrected
421 word(s) corrected
422 word(s) corrected
423 word(s) corrected
424 word(s) corrected
425 word(s) corrected
426 word(s) corrected
427 word(s

758 word(s) corrected
759 word(s) corrected
760 word(s) corrected
761 word(s) corrected
762 word(s) corrected
763 word(s) corrected
764 word(s) corrected
765 word(s) corrected
766 word(s) corrected
767 word(s) corrected
768 word(s) corrected
769 word(s) corrected
770 word(s) corrected
771 word(s) corrected
772 word(s) corrected
773 word(s) corrected
774 word(s) corrected
775 word(s) corrected
776 word(s) corrected
777 word(s) corrected
778 word(s) corrected
779 word(s) corrected
780 word(s) corrected
781 word(s) corrected
782 word(s) corrected
783 word(s) corrected
784 word(s) corrected
785 word(s) corrected
786 word(s) corrected
787 word(s) corrected
788 word(s) corrected
789 word(s) corrected
790 word(s) corrected
791 word(s) corrected
792 word(s) corrected
793 word(s) corrected
794 word(s) corrected
795 word(s) corrected
796 word(s) corrected
797 word(s) corrected
798 word(s) corrected
799 word(s) corrected
800 word(s) corrected
801 word(s) corrected
802 word(s) corrected
803 word(s

1128 word(s) corrected
1129 word(s) corrected
1130 word(s) corrected
1131 word(s) corrected
1132 word(s) corrected
1133 word(s) corrected
1134 word(s) corrected
1135 word(s) corrected
1136 word(s) corrected


From this list we can find all of the False Postivies which are the words which could not be found in the corpus but were already spelled correctly. This means that for each tuple the first 2 elements will match but the third will differ to a similar word (with smallest edit distance and highest unigram prob).

In [57]:
False_Pos = []
for trip in trips:
    if trip[0] == trip[1] and trip[0] != trip[2]:
        False_Pos.append(trip)

# We can then get the frequency distribution of the false positives
Freq_FP = nltk.FreqDist(False_Pos)
Freq_FP.most_common(15)


[(('s', 's', '.'), 5),
 (('bullimore', 'bullimore', 'bullock'), 5),
 (('flogger', 'flogger', 'longer'), 4),
 (('charge', 'charge', 'charged'), 4),
 (('smallerden', 'smallerden', 'called'), 3),
 (('i.t.v', 'i.t.v', 't.v'), 3),
 (('soap', 'soap', 'so'), 3),
 (('tuesday', 'tuesday', 'thursday'), 2),
 (('clock', 'clock', 'cloak'), 2),
 (('anglia', 'anglia', 'again'), 2),
 (('ringing', 'ringing', 'singing'), 2),
 (('rope', 'rope', 'hope'), 2),
 (('plain', 'plain', 'plan'), 2),
 (('grandsire', 'grandsire', 'handsome'), 2),
 (('sty', 'sty', 'say'), 2)]

A solution to these false positives would be to increase the corpus size so that more of these common words would
be predicted.  
However, from the top 6 most common incorrect predictions 3 of which are proper nouns which I believe another corpus would
struggle to predict.  Also, at the moment we have a corpus that is based on the same subject matter and so would
contain some proper nouns and would predict them correctly. This leads me to believe my best option would be to add to this corpus.
***
The next step would be to see the other side of the incorrect predictions i.e the mispelled words that were still incorrectly predicted. This would mean that none of the elements in the tuple are the same.


In [15]:
other_errors = []
for trip in trips:
    if trip[0] != trip[1] and trip[0] != trip[2]:
        other_errors.append(trip)

# We can then get the frequency distribution of the false positives
Freq_OE = nltk.FreqDist(other_errors)
Freq_OE.most_common(15)

[(('watch', 'wach', 'each'), 4),
 (('her', 'here', 'here'), 4),
 (('too', 'to', 'to'), 4),
 (('gave', 'cave', 'have'), 4),
 (('killed', 'kild', 'kill'), 3),
 (('what', 'wat', 'was'), 3),
 (('eyes', 'iyes', 'yes'), 2),
 (('knocked', 'nock', 'knock'), 2),
 (('wheel', 'weel', 'well'), 2),
 (('gas', 'gass', 'grass'), 2),
 (('masks', 'marsks', 'mark'), 2),
 (('soap', 'sope', 'some'), 2),
 (('sure', 'shore', 'shore'), 2),
 (('worm', 'worme', 'worms'), 2),
 (('fattening', 'fating', 'waiting'), 2)]

There seems to be three main reasons for this error: 
> - The first is similar to above the correct spelling of the word is not in the corpus.
>- The second is that we don't believe our distance metric is sufficient as mispellings could lead to creating other words for example _('her', 'here', 'here')_ and _('too', 'to', 'to')_ or the mispelling could be "closer" to another word for example _('killed', 'kild', 'kill')_
>- The final reason is that the unigram probability is choosing the incorrect word to replace the mispelled word for example _('watch', 'wach', 'each')_ in this case both watch and each are equidistant from the word 'wach' however, the word each is far more common and so is chosen as words are considered alone without context. Another example is _('eyes', 'iyes', 'yes')_ 
    

### False Positives Issue

There is no way that I can predict those proper nouns from th False Positives although I can attempt to correct the others such as 'Tuesday' . To see if this works I will need to redefine most or my original functions to include another corpus in which each word is appended to the old. I have chosen the brown corpus.

In [5]:
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\josho\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [6]:
from nltk.corpus import brown

The below code will tell me what percentage of my false positive words are present in the brown corpus and from this we can see that ther is about 42%.

In [56]:
present = []
for fp in False_Pos:
    if fp[0] in brown.words():
        present.append(1)
    else:
        present.append(0)
sum(present)/len(present)

0.41935483870967744

In [12]:
def unigram_improved(word):
    # Write your code here.
    vocab = []
    for el in train:
        for char in el['corrected']:
            vocab.append(char.lower())
            
    for token in brown.words():
        vocab.append(token)
        
    freq_dist = nltk.FreqDist(vocab)
    return freq_dist[word]

def prob_improved(word):
    # Write your code here.
    vocab = []
    for el in train:
        for char in el['corrected']:
            vocab.append(char.lower())
    
    for token in brown.words():
        vocab.append(token)
    
    return unigram_improved(word)/len(vocab)

def get_candidates_improved(token):
    # Write your code here.
    vocab = []
    close_tokens = []
    
    min_dist = -1
    
    for el in train:
        for char in el['corrected']:
            vocab.append(char.lower())
    
    for tokens in brown.words():
        vocab.append(tokens.lower())
    
    vocab_unique = set(vocab)
    
    
    for word in vocab_unique:
        distance = edit_distance(token, word)
        
        if min_dist < 0:
            min_dist = distance
        else:
            
            if distance < min_dist:
                min_dist = distance
            else:
                pass
        
    for word in vocab_unique:
        distance = edit_distance(token, word)
        
        if distance == min_dist:
            close_tokens.append(word)
        
    return close_tokens

def correct_improved(sentence):
    # Write your code here
    corrected = []
    n = 0
    vocab = []
    
    for el in train:
        for char in el['corrected']:
            vocab.append(char.lower())
            
    for token in brown.words():
        vocab.append(token)
                
    vocab_unique = list(set(vocab))
    #tokens = nltk.word_tokenize(sentence)
    for token in sentence:
        probs = []
        token = token.lower()
        if token in vocab_unique:
            corrected.append(token)
        else:
            candidates = get_candidates_improved(token)
            for word in candidates:
                probs.append(prob(word))
            corrected.append(candidates[probs.index(max(probs))])
            
        n += 1
        print(f'{n} word(s) corrected')
    return corrected

def accuracy_improved(test):
    # Write your code here
    correction = []
    false = []
    true =[]
    pairs = []
    match = []
    match_unique = []
    
    
    for el in test:
        for char in el['original']:
            false.append(char.lower())
    print(f'the length of the false string is {len(false)}')
    
            
    correction = correct_improved(false)
    
    
    for el in test:
        for char in el['corrected']:
            true.append(char.lower())
    print(f'the length of the true string is {len(true)}')
    
            
    zipped = zip(true,correction)
    for pair in zipped:
        pairs.append(pair)
    #print(pairs)
    
    for pair in pairs:
        if pair[0] == pair[1]:
            match.append(1)
        else:
            match.append(0)        
    
    pairs_unique = set(pairs)
        
    for pair in pairs_unique:
        if pair[0] == pair[1]:
            match_unique.append(1)
        else:
            match_unique.append(0)
        
    
    
    print(f'The accuracy of this model when counting every word is {(sum(match)/len(match)) * 100}%')
    print(f'The accuracy of this model when counting only unique words is {(sum(match_unique)/len(match_unique)) * 100}%')


In [46]:
accuracy(test)

the length of the false string is 1136
1 word(s) corrected
2 word(s) corrected
3 word(s) corrected
4 word(s) corrected
5 word(s) corrected
6 word(s) corrected
7 word(s) corrected
8 word(s) corrected
9 word(s) corrected
10 word(s) corrected
11 word(s) corrected
12 word(s) corrected
13 word(s) corrected
14 word(s) corrected
15 word(s) corrected
16 word(s) corrected
17 word(s) corrected
18 word(s) corrected
19 word(s) corrected
20 word(s) corrected
21 word(s) corrected
22 word(s) corrected
23 word(s) corrected
24 word(s) corrected
25 word(s) corrected
26 word(s) corrected
27 word(s) corrected
28 word(s) corrected
29 word(s) corrected
30 word(s) corrected
31 word(s) corrected
32 word(s) corrected
33 word(s) corrected
34 word(s) corrected
35 word(s) corrected
36 word(s) corrected
37 word(s) corrected
38 word(s) corrected
39 word(s) corrected
40 word(s) corrected
41 word(s) corrected
42 word(s) corrected
43 word(s) corrected
44 word(s) corrected
45 word(s) corrected
46 word(s) corrected
47 w

382 word(s) corrected
383 word(s) corrected
384 word(s) corrected
385 word(s) corrected
386 word(s) corrected
387 word(s) corrected
388 word(s) corrected
389 word(s) corrected
390 word(s) corrected
391 word(s) corrected
392 word(s) corrected
393 word(s) corrected
394 word(s) corrected
395 word(s) corrected
396 word(s) corrected
397 word(s) corrected
398 word(s) corrected
399 word(s) corrected
400 word(s) corrected
401 word(s) corrected
402 word(s) corrected
403 word(s) corrected
404 word(s) corrected
405 word(s) corrected
406 word(s) corrected
407 word(s) corrected
408 word(s) corrected
409 word(s) corrected
410 word(s) corrected
411 word(s) corrected
412 word(s) corrected
413 word(s) corrected
414 word(s) corrected
415 word(s) corrected
416 word(s) corrected
417 word(s) corrected
418 word(s) corrected
419 word(s) corrected
420 word(s) corrected
421 word(s) corrected
422 word(s) corrected
423 word(s) corrected
424 word(s) corrected
425 word(s) corrected
426 word(s) corrected
427 word(s

758 word(s) corrected
759 word(s) corrected
760 word(s) corrected
761 word(s) corrected
762 word(s) corrected
763 word(s) corrected
764 word(s) corrected
765 word(s) corrected
766 word(s) corrected
767 word(s) corrected
768 word(s) corrected
769 word(s) corrected
770 word(s) corrected
771 word(s) corrected
772 word(s) corrected
773 word(s) corrected
774 word(s) corrected
775 word(s) corrected
776 word(s) corrected
777 word(s) corrected
778 word(s) corrected
779 word(s) corrected
780 word(s) corrected
781 word(s) corrected
782 word(s) corrected
783 word(s) corrected
784 word(s) corrected
785 word(s) corrected
786 word(s) corrected
787 word(s) corrected
788 word(s) corrected
789 word(s) corrected
790 word(s) corrected
791 word(s) corrected
792 word(s) corrected
793 word(s) corrected
794 word(s) corrected
795 word(s) corrected
796 word(s) corrected
797 word(s) corrected
798 word(s) corrected
799 word(s) corrected
800 word(s) corrected
801 word(s) corrected
802 word(s) corrected
803 word(s

1135 word(s) corrected
1136 word(s) corrected
the length of the true string is 1136
The accuracy of this model when counting every word is 85.03521126760563%
The accuracy of this model when counting only unique words is 70.93333333333334%


### False positives Results
*** 
From the above cells it can be seen that I implemented a solution for false positives which was to add to the already existing corpus. I added the entire brown corpus as from looking at each category individually the max coverage was from the fiction category in which it contained about 30% of the false positives whereas the entire corpus contained about 42% of the words.
  
  Although the above cell contains almost 140 lines of code this really translates to about 5 lines of new code however each of the original functions had to be redefined.
  From this solution now more than 85% of all words are predicted correctly and almost 71% of unique words are predicted correctly which is an improvement from the 84% and 67% previously.
  
  It is worth noting that this addition is much more computationally heavy however, I believe it still yields a net benefit due to its advantages when I speak about the next improvement.

## Incorrect words being incorrectly predicted
***
As I previously mentioned a percentage of these errors are due to the fact that the correct word soes not exist in the corpus and the above solution should address this.  

  The next issue I began to address is the limitation of using the unigram probability function. The main limitation is that this function ignore the context of the sentence and simply chooses the most common word in the corpus. The next improvement I would make would be to use a bigram or trigram model such that the context is considered. I will still however be using the edit_distance function to select the original candidates.
  

In [18]:
vocab = []
for element in train:
    for word in element['corrected']:
        vocab.append(word)
   
 #vocab.append(element['corrected'])
for token in brown.words():
        vocab.append(token)

bigram =  [t for t in nltk.bigrams(vocab)]
def bigram_prob(phrase):
    pairs_with_w2 = []
    pairs_with_both = []
    for pair in bigram:
        if phrase[1].lower() == pair[1].lower():
            pairs_with_w2.append(pair)
            
    total = len(pairs_with_w2)
    
    for pair in pairs_with_w2:
        if phrase[0].lower() == pair[0].lower():
            pairs_with_both.append(pair)
            
    x = len(pairs_with_both)
    #print(pairs_with_w2)
    
    return x/total


In [102]:
def correct_bigram(phrase):
    unique_vocab = list(set(vocab))
    corrected = []
    
    bigram_sentence = bigram =  [t for t in nltk.bigrams(phrase)]

    for word in phrase:
        candidates = []
        probs = []
        if phrase.index(word) == 0:
            if word.lower() in unique_vocab:
                #print(word)
                corrected.append(word.lower())
            else:
                candidates = get_candidates_improved(word.lower())
                #print(candidates)
                for candidate in candidates:
                    probs.append(prob_improved(candidate))
                corrected.append(candidates[probs.index(max(probs))]) 
        else:
            if word.lower() in unique_vocab:
                corrected.append(word.lower())
            else:
                candidates = get_candidates_improved(word.lower())
                for candidate in candidates:
                    probs.append(bigram_prob([corrected[-1].lower(), candidate.lower()]))
                corrected.append(candidates[probs.index(max(probs))])
        print(corrected)
    return corrected
    
            

In [105]:
def bigram_accuracy(data):
    correct_sentences = []
    correct_words = []
    corrected_sentences = []
    corrected_words = []
    incorrect_sentences = []
    pairs = []
    matches = []
    matches_unique = []
    n = 0
    
    for sentence in data:
        correct_sentences.append(sentence['corrected'])
        
    for sentence in correct_sentences:
        for word in sentence:
            correct_words.append(word.lower())
            
    for sentence in data:
        incorrect_sentences.append(sentence['original'])
    print(len(incorrect_sentences))
        
    for sentence in incorrect_sentences:
        corrected_sentences.append(correct_bigram(sentence))
        n += 1
        print(n)
        
    for sentence in corrected_sentences:
        for word in sentence:
            corrected_words.append(word.lower())
            
    zipped = zip(correct_words, corrected_words)
    for pair in zipped:
        pairs.append(pair)
        
    pairs_unique = set(pairs)
    
    for pair in pairs:
        if pair[0] == pair[1]:
            matches.append(1)
        else:
            matches.append(0)
            
    for pair in pairs_unique:
        if pair[0] == pair[1]:
            matches_unique.append(1)
        else:
            matches_unique.append(0)
    print(pairs)
    print(f'The accuracy of this model when counting every word is {(sum(matches)/len(matches)) * 100}%')
    print(f'The accuracy of this model when counting only unique words is {(sum(matches_unique)/len(matches_unique)) * 100}%')

In [106]:
tist = test[:40]
bigram_accuracy(tist)

40
['1']
['1', '.']
1
['niger']
['niger', 'thrush']
['niger', 'thrush', 'page']
['niger', 'thrush', 'page', '48']
2
['i']
['i', 'have']
['i', 'have', 'four']
['i', 'have', 'four', 'in']
['i', 'have', 'four', 'in', 'my']
['i', 'have', 'four', 'in', 'my', 'family']
['i', 'have', 'four', 'in', 'my', 'family', 'dad']
['i', 'have', 'four', 'in', 'my', 'family', 'dad', 'mum']
['i', 'have', 'four', 'in', 'my', 'family', 'dad', 'mum', 'and']
['i', 'have', 'four', 'in', 'my', 'family', 'dad', 'mum', 'and', 'sister']
['i', 'have', 'four', 'in', 'my', 'family', 'dad', 'mum', 'and', 'sister', '.']
3
['my']
['my', 'dad']
['my', 'dad', 'works']
['my', 'dad', 'works', 'at']
['my', 'dad', 'works', 'at', 'melton']
['my', 'dad', 'works', 'at', 'melton', '.']
4
['my']
['my', 'sister']
['my', 'sister', 'go']
['my', 'sister', 'go', 'to']
['my', 'sister', 'go', 'to', 'tonbury']
['my', 'sister', 'go', 'to', 'tonbury', '.']
5
['my']
['my', 'mum']
['my', 'mum', 'goes']
['my', 'mum', 'goes', 'out']
['my', 'mum'

['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', 'to']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', 'to', 'work']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', 'to', 'work', 'at']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', 'to', 'work', 'at', 'smaller']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', '

['the']
['the', 'name']
['the', 'name', 'of']
['the', 'name', 'of', 'the']
['the', 'name', 'of', 'the', 'man']
['the', 'name', 'of', 'the', 'man', 'who']
['the', 'name', 'of', 'the', 'man', 'who', 'was']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he', 'has']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he', 'has', 'black']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he', 'has', 'black', 'hair']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he', 'has', 'black

['i']
['i', 'kissed']
['i', 'kissed', 'a']
['i', 'kissed', 'a', 'girl']
['i', 'kissed', 'a', 'girl', 'one']
['i', 'kissed', 'a', 'girl', 'one', 'night']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue', 'she']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue', 'she', 'said']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue', 'she', 'said', 'o']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue', 'she', 'said', 'o', 'do']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burnin

In [107]:
bigram_accuracy(test)

100
['1']
['1', '.']
1
['niger']
['niger', 'thrush']
['niger', 'thrush', 'page']
['niger', 'thrush', 'page', '48']
2
['i']
['i', 'have']
['i', 'have', 'four']
['i', 'have', 'four', 'in']
['i', 'have', 'four', 'in', 'my']
['i', 'have', 'four', 'in', 'my', 'family']
['i', 'have', 'four', 'in', 'my', 'family', 'dad']
['i', 'have', 'four', 'in', 'my', 'family', 'dad', 'mum']
['i', 'have', 'four', 'in', 'my', 'family', 'dad', 'mum', 'and']
['i', 'have', 'four', 'in', 'my', 'family', 'dad', 'mum', 'and', 'sister']
['i', 'have', 'four', 'in', 'my', 'family', 'dad', 'mum', 'and', 'sister', '.']
3
['my']
['my', 'dad']
['my', 'dad', 'works']
['my', 'dad', 'works', 'at']
['my', 'dad', 'works', 'at', 'melton']
['my', 'dad', 'works', 'at', 'melton', '.']
4
['my']
['my', 'sister']
['my', 'sister', 'go']
['my', 'sister', 'go', 'to']
['my', 'sister', 'go', 'to', 'tonbury']
['my', 'sister', 'go', 'to', 'tonbury', '.']
5
['my']
['my', 'mum']
['my', 'mum', 'goes']
['my', 'mum', 'goes', 'out']
['my', 'mum

['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', 'to']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', 'to', 'work']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', 'to', 'work', 'at']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', 'to', 'work', 'at', 'smaller']
['my', 'mum', 'is', 'at', 'home', 'she', 'goes', 'to', 'the', 'shop', 'on', 'fridays', 'my', 'dad', 'goes', '

['the']
['the', 'name']
['the', 'name', 'of']
['the', 'name', 'of', 'the']
['the', 'name', 'of', 'the', 'man']
['the', 'name', 'of', 'the', 'man', 'who']
['the', 'name', 'of', 'the', 'man', 'who', 'was']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he', 'has']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he', 'has', 'black']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he', 'has', 'black', 'hair']
['the', 'name', 'of', 'the', 'man', 'who', 'was', 'wild', 'was', 'jack', 'robinson', 'he', 'has', 'black

['i']
['i', 'kissed']
['i', 'kissed', 'a']
['i', 'kissed', 'a', 'girl']
['i', 'kissed', 'a', 'girl', 'one']
['i', 'kissed', 'a', 'girl', 'one', 'night']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue', 'she']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue', 'she', 'said']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue', 'she', 'said', 'o']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burning', 'blue', 'she', 'said', 'o', 'do']
['i', 'kissed', 'a', 'girl', 'one', 'night', 'here', 'eyes', 'were', 'burnin

['me']
['me', 'and']
['me', 'and', 'my']
['me', 'and', 'my', 'dad']
['me', 'and', 'my', 'dad', 'bike']
['me', 'and', 'my', 'dad', 'bike', 'to']
['me', 'and', 'my', 'dad', 'bike', 'to', 'melton']
['me', 'and', 'my', 'dad', 'bike', 'to', 'melton', '.']
45
['the']
['the', 'sick']
['the', 'sick', 'sow']
['the', 'sick', 'sow', 'of']
['the', 'sick', 'sow', 'of', 'the']
['the', 'sick', 'sow', 'of', 'the', 'army']
['the', 'sick', 'sow', 'of', 'the', 'army', 'court']
46
['one']
['one', 'day']
['one', 'day', 'sargent']
['one', 'day', 'sargent', 's']
['one', 'day', 'sargent', 's', '.']
['one', 'day', 'sargent', 's', '.', 'm']
['one', 'day', 'sargent', 's', '.', 'm', '.']
['one', 'day', 'sargent', 's', '.', 'm', '.', 'baltimore']
['one', 'day', 'sargent', 's', '.', 'm', '.', 'baltimore', 'told']
['one', 'day', 'sargent', 's', '.', 'm', '.', 'baltimore', 'told', 'hut']
['one', 'day', 'sargent', 's', '.', 'm', '.', 'baltimore', 'told', 'hut', '29']
['one', 'day', 'sargent', 's', '.', 'm', '.', 'balt

['get']
['get', 'back']
['get', 'back', 'in']
['get', 'back', 'in', 'the']
['get', 'back', 'in', 'the', 'line']
['get', 'back', 'in', 'the', 'line', 'before']
['get', 'back', 'in', 'the', 'line', 'before', 'i']
['get', 'back', 'in', 'the', 'line', 'before', 'i', 'put']
['get', 'back', 'in', 'the', 'line', 'before', 'i', 'put', 'you']
['get', 'back', 'in', 'the', 'line', 'before', 'i', 'put', 'you', 'on']
['get', 'back', 'in', 'the', 'line', 'before', 'i', 'put', 'you', 'on', 'charge']
55
['what']
['what', 'did']
['what', 'did', 'you']
['what', 'did', 'you', 'say']
56
['o']
['o', 'nothing']
['o', 'nothing', 'sir']
57
['go']
['go', 'and']
['go', 'and', 'get']
['go', 'and', 'get', 'some']
['go', 'and', 'get', 'some', 'wilkey']
58
['okay']
['okay', 'sir']
59
['put']
['put', 'some']
['put', 'some', 'soap']
['put', 'some', 'soap', 'in']
['put', 'some', 'soap', 'in', 'it']
['put', 'some', 'soap', 'in', 'it', 'flogged']
['put', 'some', 'soap', 'in', 'it', 'flogged', 'said']
['put', 'some', 'so

['when', 'you', 'clean', 'them', 'out', 'you', 'should', 'not', 'leave', 'a', 'walk', 'in', 'with', 'them', 'because', 'the', 'mother', 'might', 'knock']
['when', 'you', 'clean', 'them', 'out', 'you', 'should', 'not', 'leave', 'a', 'walk', 'in', 'with', 'them', 'because', 'the', 'mother', 'might', 'knock', 'it']
['when', 'you', 'clean', 'them', 'out', 'you', 'should', 'not', 'leave', 'a', 'walk', 'in', 'with', 'them', 'because', 'the', 'mother', 'might', 'knock', 'it', 'down']
['when', 'you', 'clean', 'them', 'out', 'you', 'should', 'not', 'leave', 'a', 'walk', 'in', 'with', 'them', 'because', 'the', 'mother', 'might', 'knock', 'it', 'down', 'and']
['when', 'you', 'clean', 'them', 'out', 'you', 'should', 'not', 'leave', 'a', 'walk', 'in', 'with', 'them', 'because', 'the', 'mother', 'might', 'knock', 'it', 'down', 'and', 'the']
['when', 'you', 'clean', 'them', 'out', 'you', 'should', 'not', 'leave', 'a', 'walk', 'in', 'with', 'them', 'because', 'the', 'mother', 'might', 'knock', 'it', '

['we', 'give', 'them', 'some', 'dating', 'food', 'called', 'somber']
['we', 'give', 'them', 'some', 'dating', 'food', 'called', 'somber', '2']
88
['the']
['the', 'you']
['the', 'you', 'mean']
['the', 'you', 'mean', 'that']
['the', 'you', 'mean', 'that', 'is']
['the', 'you', 'mean', 'that', 'is', 'take']
['the', 'you', 'mean', 'that', 'is', 'take', 'them']
['the', 'you', 'mean', 'that', 'is', 'take', 'them', 'from']
['the', 'you', 'mean', 'that', 'is', 'take', 'them', 'from', 'their']
['the', 'you', 'mean', 'that', 'is', 'take', 'them', 'from', 'their', 'mother']
['the', 'you', 'mean', 'that', 'is', 'take', 'them', 'from', 'their', 'mother', 'you']
['the', 'you', 'mean', 'that', 'is', 'take', 'them', 'from', 'their', 'mother', 'you', 'have']
['the', 'you', 'mean', 'that', 'is', 'take', 'them', 'from', 'their', 'mother', 'you', 'have', 'got']
['the', 'you', 'mean', 'that', 'is', 'take', 'them', 'from', 'their', 'mother', 'you', 'have', 'got', 'to']
['the', 'you', 'mean', 'that', 'is', 't

['my', 'heart', 'is', 'full', 'of', 'sadness', 'my', 'heart', 'is', 'full', 'of', 'joy', 'it', 'might', 'be', 'my', 'wife', 'or', 'it', 'might', 'be', 'helen']
['my', 'heart', 'is', 'full', 'of', 'sadness', 'my', 'heart', 'is', 'full', 'of', 'joy', 'it', 'might', 'be', 'my', 'wife', 'or', 'it', 'might', 'be', 'helen', 'of']
['my', 'heart', 'is', 'full', 'of', 'sadness', 'my', 'heart', 'is', 'full', 'of', 'joy', 'it', 'might', 'be', 'my', 'wife', 'or', 'it', 'might', 'be', 'helen', 'of', 'toy']
98
['one']
['one', 'saturday']
['one', 'saturday', 'i']
['one', 'saturday', 'i', 'though']
['one', 'saturday', 'i', 'though', 'i']
['one', 'saturday', 'i', 'though', 'i', 'would']
['one', 'saturday', 'i', 'though', 'i', 'would', 'go']
['one', 'saturday', 'i', 'though', 'i', 'would', 'go', 'to']
['one', 'saturday', 'i', 'though', 'i', 'would', 'go', 'to', 'the']
['one', 'saturday', 'i', 'though', 'i', 'would', 'go', 'to', 'the', 'races']
['one', 'saturday', 'i', 'though', 'i', 'would', 'go', 'to',

from the above I have outputted each word being corrected over time to aid with debugging.  I also included a list of each pair being compared to see which words were difficult to predict.  
Finally I generated the accuracy when considering duplicates and when not considering duplicates.
  The accuracy when considering duplicates is almost 90% and when not considering duplicates is 76% which are my best values yet.

## Results for N-gram model
***
Above I have implemented a bigram model over a sample of my data. As this method is computationally heavy and still requires some debugging I cannot see the overall change in accuracy. I can assume that there is an improvement however as it corrects some of the original errors for example 'wach' will now be corrected to 'watch' when it is preceeded by the word 'the' when it was previously being changed to 'each'. The issue with using N-gram model is that we depend on co-occurences of the words which can be difficult in sparse data which is why I decided to leave in the entire brown corpus in my model. This could also be increased to a trigram model to improve accuracy as 'wach' is now being changed to 'wash' when preceeded by 'I' rather than the correct value 'watch'.  
  
  Also, although there are many lines of code, I have leveraged a lot of previous code in my functions.

## Other possible improvements

### PoS Tagging
***
An improvement which I did not implement but I believe could have made my model more accurate would be to use the Pos tags of the words. This will increase the amount of context considered when making predictions. Below an attempt at generating the HMM transission and emission matrices can be seen. I believe this would help some issues for example when 'iyes' is corrected to 'yes' rather than 'eyes' as both words have different Pos tags it should be clear which of the two is next in the sentence. This would of course be combined with the bigram or trigram model.

### Distance Metric
***
I found that a source of some of my errors came from using the edit_distance as my didtance metric. This can be seen with the error 'kild' which is corrected to 'kill' with an edit distance of 1 rather than the true value 'killed' woth an edit distance of 2. Wordnet could have been leveraged to select a better set of candidates.

## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

The below values can be found in their relevant sections.

### The accuracy of the original model is :

In [ ]:
accuracy(test)

### By increasing the corpus size it is:

In [ ]:
accuracy_improved(test)

### Using bigrams with the larger corpus:

In [ ]:
bigram_accuracy(test)